In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class ResidualBlock(nn.Module):
  def __init__(self, channels):
    super(ResidualBlock, self).__init__()
    self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

  def forward(self, x):
    y = F.relu(self.conv1(x))
    y = self.conv2(x)
    output = F.relu(x+y)
    return output

In [3]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
    self.mp = nn.MaxPool2d(2)

    self.rblock1 = ResidualBlock(16)
    self.rblock2 = ResidualBlock(32)

    # self.fc = nn.Linear(512, 10)   # self.fc = nn.Linear(.., 10)


  def forward(self, x):
    in_size = x.size(0)
    x = self.mp(F.relu(self.conv1(x)))
    x = self.rblock1(x)
    x = self.mp(F.relu(self.conv2(x)))
    x = self.rblock2(x)
    x = x.view(in_size, -1)
    # x = self.fc(x)    #omit this part at the begining
    return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (rblock1): ResidualBlock(
    (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (rblock2): ResidualBlock(
    (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)


In [4]:
import torchvision.models as models
from torchsummary import summary

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)
summary(net, (1, 28, 28))    #shows the flow of the network, from the last line can infer the output should be 32*4*4

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 24, 24]             416
         MaxPool2d-2           [-1, 16, 12, 12]               0
            Conv2d-3           [-1, 16, 12, 12]           2,320
            Conv2d-4           [-1, 16, 12, 12]           2,320
     ResidualBlock-5           [-1, 16, 12, 12]               0
            Conv2d-6             [-1, 32, 8, 8]          12,832
         MaxPool2d-7             [-1, 32, 4, 4]               0
            Conv2d-8             [-1, 32, 4, 4]           9,248
            Conv2d-9             [-1, 32, 4, 4]           9,248
    ResidualBlock-10             [-1, 32, 4, 4]               0
Total params: 36,384
Trainable params: 36,384
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.17
Params size (MB): 0.14
Estimated Tot